In [1]:
!pip install keras_tuner

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from tensorflow import keras
import keras_tuner as kt


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("train input shape", x_train.shape)
print("train label shape", y_train.shape)
print("test input shape", x_test.shape)
print("test label shape", y_test.shape)

# normalize pixels
x_train = x_train.astype("float32")  / 255
x_test = x_test.astype("float32")  / 255

# encode labels with one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

170508288/170498071 [==============================] - 2s 0us/step
train input shape (50000, 32, 32, 3)
train label shape (50000, 1)
test input shape (10000, 32, 32, 3)
test label shape (10000, 1)


In [6]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.losses import CategoricalCrossentropy

def build_model(hp):  
    p = 0.2
    model = keras.models.Sequential()
    model.add(Conv2D(hp.Int('conv1_filters', min_value=32, max_value=256, step=32),
                    (3, 3),
                    input_shape=x_train.shape[1:],
                    activation=hp.Choice("activation", ["relu", "tanh"]))) 
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # tune number of convolutional layers (at least 2, at most 5)
    # and also number of filters (at least 32, at most 256)
    # and activation function (relu or tanh)
    # and whether use dropout
    for i in range(hp.Int('conv_layers', 2, 5)):
        kernel_size = hp.Choice(f'kernel_size{i}', values=[3, 5])
        model.add(Conv2D(hp.Int(f'conv{i}_filters', min_value=32, max_value=256, step=32),
                         kernel_size,
                         activation=hp.Choice(f'activation{i}', ["relu", "tanh"]))) 
        if hp.Boolean("dropout"):
            model.add(Dropout(p))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(units=hp.Int(f'dense_neurons1', min_value=32, max_value=256, step=64), activation='relu'))
    
    for i in range(hp.Int('dense_layers', 2, 5)):
        model.add(Dense(units=hp.Int(f'neurons{i}', min_value=32, max_value=256, step=64), activation='relu'))

    # last layer has 10 neurons
    model.add(Dense(10, activation='softmax'))
    # tune learning rate
    alpha = hp.Choice('learning_rate', values=[0.001, 0.0005, 0.0001])
    # tune optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'SGD'])
    if optimizer == 'adam':
          optimizer = tf.keras.optimizers.Adam(learning_rate=alpha)
    else: optimizer = tf.keras.optimizers.SGD(learning_rate=alpha)
    model.compile(optimizer=optimizer,
                  loss=CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

build_model(kt.HyperParameters())

In [8]:
tuner = kt.RandomSearch(hypermodel=build_model,
                        objective="val_accuracy",
                        max_trials=4,
                        executions_per_trial=3,
                        overwrite=True,
                        directory="result",
                        project_name="cnn_tuning",
                        )

tuner.search_space_summary()

Search space summary
Default search space size: 15
conv1_filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
kernel_size0 (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
conv0_filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
activation0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
kernel_size1 (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
activation1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dense_neurons1 (Int)
{'default': None, 'con

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [10]:
tuner.search(x_train, y_train, epochs=8, validation_data=(x_test, y_test), callbacks=[stop_early])

models = tuner.get_best_models(num_models=1)
best_model = models[0]

Trial 4 Complete [00h 03m 56s]
val_accuracy: 0.11963333189487457

Best val_accuracy So Far: 0.6780666510264078
Total elapsed time: 00h 21m 35s


In [11]:
best_model.build(input_shape=(None, 32, 32, 3))
print(best_model.summary())
loss, accuracy = best_model.evaluate(x_test, y_test)

print('loss:', loss)
print('accuracy:', accuracy)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 224)       1433824   
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 224)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 256)         516352    
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 9, 256)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 256)         0

In [12]:
tuner.results_summary()

Results summary
Results in result/cnn_tuning
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv1_filters: 256
activation: tanh
conv_layers: 2
kernel_size0: 5
conv0_filters: 224
activation0: relu
dropout: True
kernel_size1: 3
activation1: tanh
dense_neurons1: 224
dense_layers: 3
neurons0: 224
neurons1: 96
learning_rate: 0.0005
optimizer: adam
neurons2: 32
Score: 0.6780666510264078
Trial summary
Hyperparameters:
conv1_filters: 96
activation: relu
conv_layers: 3
kernel_size0: 5
conv0_filters: 224
activation0: relu
dropout: True
kernel_size1: 3
activation1: relu
dense_neurons1: 160
dense_layers: 5
neurons0: 96
neurons1: 96
learning_rate: 0.0005
optimizer: SGD
neurons2: 32
kernel_size2: 3
conv2_filters: 32
activation2: relu
neurons3: 32
neurons4: 32
Score: 0.15159999827543894
Trial summary
Hyperparameters:
conv1_filters: 128
activation: relu
conv_layers: 3
kernel_size0: 3
conv0_filters: 96
activation0: relu
dropout: False
kernel_size1: